<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Gather-Follower-&amp;-Statuses-Counts-for-Podcasts-with-Twitter-Accounts" data-toc-modified-id="Gather-Follower-&amp;-Statuses-Counts-for-Podcasts-with-Twitter-Accounts-1">Gather Follower &amp; Statuses Counts for Podcasts with Twitter Accounts</a></span><ul class="toc-item"><li><span><a href="#Goal-is-to-use-these-to-enrich-our-understanding-of-podcast-social-activity." data-toc-modified-id="Goal-is-to-use-these-to-enrich-our-understanding-of-podcast-social-activity.-1.1">Goal is to use these to enrich our understanding of podcast social activity.</a></span></li></ul></li></ul></div>

In [3]:
import pickle

import features

import time
import re

import twitter
api = twitter.Api(consumer_key='')

In [4]:
print(api.VerifyCredentials())

{"created_at": "Mon Aug 26 16:59:13 +0000 2019", "default_profile": true, "description": "data scientist | meditator | Coloradan in BK/NYC", "favourites_count": 1, "followers_count": 4, "friends_count": 20, "id": 1166032070667202565, "id_str": "1166032070667202565", "name": "Thomas Kavanagh", "profile_background_color": "F5F8FA", "profile_image_url": "http://pbs.twimg.com/profile_images/1166032951932403714/lkQcnMwS_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/1166032951932403714/lkQcnMwS_normal.jpg", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "ThomasK26635081"}


## Gather Follower & Statuses Counts for Podcasts with Twitter Accounts

### Goal is to use these to enrich our understanding of podcast social activity.

Tutorial found here:

https://www.alexkras.com/how-to-get-user-feed-with-twitter-api-and-python/

We will unload (any) version of the sanitized dataframe, in order to gather a list of channels in order to create a dictionary.

In [6]:
df_version = 'merged_10-08_13-54.pickle'
import_path = '../scraped/merged/' + df_version
with open(import_path, 'rb') as file:
    raw_df = pickle.load(file)

raw_df['twitter'] = raw_df['ch_feed-socials'].apply(lambda row: features.has_domain(row, 'twitter'))

In [7]:
t_cols = ['title','chan_url','ch_feed-socials','twitter']

In [8]:
raw_df.sample(3)[t_cols]

,title,chan_url,ch_feed-socials,twitter
13,"Asian, Not Asian",https://castbox.fm/channel/Asian%2C-Not-Asian-...,[https://art19.com/shows/asian-not-asian],0
156,Psychology of Eating,https://castbox.fm/channel/Psychology-of-Eatin...,"[https://twitter.com/Psych_of_Eating, http://w...",1
25,Atomic Moms,https://castbox.fm/channel/Atomic-Moms-id125492,[https://art19.com/shows/atomic-moms],0


In [9]:
raw_df[raw_df.twitter == 1].sample(3)[t_cols]

,title,chan_url,ch_feed-socials,twitter
183,The Brutal Truth About Sales & Selling - B2B S...,https://castbox.fm/channel/The-Brutal-Truth-Ab...,"[https://twitter.com/BriangBurns, https://www....",1
118,The Robin Sharma Mastery Sessions,https://castbox.fm/channel/The-Robin-Sharma-Ma...,"[https://twitter.com/RobinSharma, https://www....",1
26,Focus on the Family Parenting Podcast,https://castbox.fm/channel/Focus-on-the-Family...,"[https://twitter.com/FocusFamily, https://www....",1


In [10]:
twitter_chans = raw_df[raw_df.twitter == 1][['title','chan_url','ch_feed-socials']].to_dict(orient='records')

In [11]:
def extract_twitter_screen_name(ch_feed_socials):

    if len(ch_feed_socials) == 0:
        return ''
    else:
        pattern = '.+(twitter.com/)(?P<screen_name>.+)'
        for url in ch_feed_socials:
#             print(url)
            try:
                screen_name = re.match(pattern, url).group('screen_name')
                if screen_name:
                    return screen_name
            except:
                oass
        return ''

In [12]:
with open('../social_metrics/twitter/channel_stats_by_name_oct7_9p.pickle', 'rb') as file:
    twitter_dict = pickle.load(file)

In [13]:
for chan in twitter_dict.keys():
    if twitter_dict[chan]['screen_name']:
        continue
    else:
        twitter_dict[chan]['screen_name'] = extract_twitter_screen_name(twitter_dict[chan]['ch_feed-socials'])

In [14]:
len(twitter_dict)

1054

Let's create a set of unique twitter handles, as its own dataframe, which we will use as a reference.

In [15]:
# with open('../social_metrics/twitter/channel_stats_by_name.pickle', 'rb') as file:
#     twitter_follower_dict = pickle.load(file)

In [16]:
def add_channels_to_twitter_dictionary(channnels, output_dict):
    for chan in channnels:
        key = chan['title']
        try:
            output_dict[key].update(chan)
        except:
            output_dict[key] = chan
    return True

In [18]:
add_channels_to_twitter_dictionary(twitter_chans, twitter_dict)

True

In [19]:
len(twitter_chans)

1612

In [20]:
[(k, twitter_dict[k]) for k in list(twitter_dict.keys())[:5]]

[('Small Town Horror',
  {'title': 'Small Town Horror',
   'chan_url': 'https://castbox.fm/channel/Small-Town-Horror-id1266396',
   'ch_feed-socials': ['https://twitter.com/SmallTwnHorror',
    'https://www.facebook.com/smalltownhorrorpodcast',
    'https://play.acast.com/s/smalltownhorror'],
   'screen_name': 'SmallTwnHorror',
   'follower_count': 0}),
 ('I Should Be Writing',
  {'title': 'I Should Be Writing',
   'chan_url': 'https://castbox.fm/channel/I-Should-Be-Writing-id210',
   'ch_feed-socials': ['https://twitter.com/mightymur',
    'http://murverse.com/tag/isbw/'],
   'screen_name': 'mightymur',
   'follower_count': 14970}),
 ('Book Riot - The Podcast',
  {'title': 'Book Riot - The Podcast',
   'chan_url': 'https://castbox.fm/channel/Book-Riot-The-Podcast-id1207097',
   'ch_feed-socials': ['https://twitter.com/BookRiot',
    'https://bookriot.com/listen/shows/thepodcast/'],
   'screen_name': 'BookRiot',
   'follower_count': 214860}),
 ('The Disney Story Origins Podcast',
  {'t

In [21]:
def update_screen_names(t_dict):
    
    chan_list = t_dict.keys()
    
    for chan in chan_list:
        if 'screen_name' in t_dict[chan].keys():
            pass
        else:
            socials = t_dict[chan]['ch_feed-socials']
            t_dict[chan]['screen_name'] = extract_twitter_screen_name(socials)
    
    return True

In [22]:
update_screen_names(twitter_dict)

True

In [23]:
[(k, twitter_dict[k]) for k in list(twitter_dict.keys())[-3:]]

[('Ram Dass Here And Now',
  {'title': 'Ram Dass Here And Now',
   'chan_url': 'https://castbox.fm/channel/Ram-Dass-Here-And-Now-id463741',
   'ch_feed-socials': ['https://twitter.com/BabaRamDass',
    'https://www.facebook.com/babaramdass',
    'https://art19.com/shows/ram-dass-here-and-now'],
   'screen_name': 'BabaRamDass'}),
 ('Gita For Daily Living',
  {'title': 'Gita For Daily Living',
   'chan_url': 'https://castbox.fm/channel/Gita-For-Daily-Living-id1329884',
   'ch_feed-socials': ['https://twitter.com/NeilRBhatt',
    'https://www.facebook.com/Gitafordailyliving',
    'https://neilbhatt.podbean.com'],
   'screen_name': 'NeilRBhatt'}),
 ('Bhagavad Gita Hindi',
  {'title': 'Bhagavad Gita Hindi',
   'chan_url': 'https://castbox.fm/channel/Bhagavad-Gita-Hindi-id464044',
   'ch_feed-socials': ['https://twitter.com/YatharthGeeta52',
    'https://www.facebook.com/bhagavadgitaclass',
    'https://audioboom.com/channel/bhagavad-gita-hindi'],
   'screen_name': 'YatharthGeeta52'})]

In [24]:
def get_follower_counts(t_dict, api_calls=1):
    
    chan_list = t_dict.keys()
    
    for chan in chan_list:
        
#         time.sleep(3)

        if api_calls < 1:
            return 'local API limit reached'
        if 'screen_name' in t_dict[chan].keys():
            sn = t_dict[chan]['screen_name']
#             print(f'about to scan {chan} with sn {sn}')
            if 'follower_count' in t_dict[chan].keys():
#                 print('already calculated follower count for ',chan)
                continue
            else:
                try:
                    user = api.GetUser(screen_name=sn)
                    follower_count = user.followers_count

                    print(f'user {sn} has {follower_count} followers')
                    t_dict[chan]['follower_count'] = follower_count

                    api_calls = api_calls - 1

                except:
                    t_dict[chan]['follower_count'] = 0
                    print(f'failed to pull follower count for {chan}')
                    continue
        else:
            continue

In [25]:
get_follower_counts(twitter_dict, api_calls=4000)

user TOSPOfficial has 14 followers
user geekgamertv has 4237 followers
user PGoRadio has 273 followers
user GeeklyInc has 7489 followers
user DMs_Block has 12616 followers
user AdvInQuesting has 919 followers
user TheCritshow has 554 followers
user theindep has 2210 followers
user GuardiansofD has 45868 followers
user BloodonThames has 839 followers
user soundsofcrowes has 363 followers
user JoelOsteen has 9094213 followers
failed to pull follower count for Exploring My Strange Bible
user NancyDeMoss has 31908 followers
user lifechurch has 59828 followers
user frbekomson has 3742 followers
user AbidingPodcast has 1278 followers
user FierceMarriage has 18425 followers
user freshlife has 9291 followers
user tgc has 265490 followers
failed to pull follower count for Comments by Celebs
failed to pull follower count for The Bellas Podcast
user TalkIsJericho has 61942 followers
user GrossesTetesRTL has 56239 followers
user BGNPodcast has 6572 followers
failed to pull follower count for Laure

user SamSeder has 120377 followers
user dccommonsense has 113338 followers
user amjoyshow has 457771 followers
user intercepted has 17712 followers
user BrianLehrer has 62026 followers
user DLoesch has 1148999 followers
user theintercept has 723372 followers
user voxdotcom has 869566 followers
user abcradio has 2 followers
user ElectionPodcast has 389 followers
user InTheThickShow has 6806 followers
user jjvirgin has 22402 followers
user EatThisNotThat has 633795 followers
user FoodHeavenShow has 4240 followers
user ATPScience1 has 669 followers
user Healthy_Couples has 832 followers
user TherapyChatPod has 1887 followers
user EricTivers has 3917 followers
user lavendaire has 11855 followers
user Fearnecotton has 7100045 followers
user Phil_Larson has 1808 followers
user brookemcalary has 3361 followers
user TheBubbleHour has 1427 followers
user HappinesPodcast has 14504 followers
user snoozecast has 28 followers
user 21fivepodcast has 7 followers
failed to pull follower count for Bedt

failed to pull follower count for House Call With Dr. Hyman
user chriskresser has 98810 followers
user OLDPodcast has 5201 followers
user groundupshow has 632 followers
user tmppodcast has 336 followers
user CTFPodcast has 2210 followers
user AffirmationPod has 786 followers
user KetoConnect has 24449 followers
user fatburnman has 32526 followers
user anxietyslayer has 593 followers
user BrockSky has 1457 followers
user LondonRealTV has 116681 followers
user Kingsbu has 37881 followers
user meghanrienks has 565630 followers
user EMCases has 8812 followers
user Psych_of_Eating has 19123 followers
user MarathonAcademy has 2597 followers
user TBBCoach411 has 82610 followers
user blackgirlinom has 23826 followers
user BalancedBlondie has 5208 followers
user DishUpNutrition has 105 followers
user BarbellShrugged has 32829 followers
user alisakeeton has 1629 followers
user _elleruss has 957 followers
user Dopeypodcast has 2296 followers
user NEJM has 612910 followers
user VinnieTortorich has

In [26]:
list(twitter_dict.keys())[-20:]

['The WAN Show Podcast',
 'Machine Learning – Software Engineering Daily',
 'MacBreak Weekly (MP3)',
 'This Week in Google (MP3)',
 'Windows Weekly (MP3)',
 'All TWiT.tv Shows (MP3)',
 'The Fully Charged Show Podcast',
 'This Week in Tech (Video HI)',
 'Koron',
 'The Great Albums',
 'In the Groove, Jazz and Beyond',
 'BBC Music Magazine',
 'Meet the Composer',
 'The Doc Hicks Podcast',
 'Pulp Voices Podcast - پادکست صداهای عامه\u200cپسند',
 'Blink-155',
 'ARTIFICIAL GHOST RADIO',
 'Ram Dass Here And Now',
 'Gita For Daily Living',
 'Bhagavad Gita Hindi']

In [27]:
with open('../social_metrics/twitter/channel_stats_by_name_oct8_2p.pickle', 'wb') as file:
    pickle.dump(twitter_dict, file)